# Processing your Data with topsApp.py

**Author**: Bekaert David - Jet Propulsion Laboratory

In this notebook, we will walk through the various steps of processing with topsApp.py. 

topsApp.py is a pair-by-pair interferometric processor that takes as input two Sentinel-1 SAR acquisitions acquired in TOPS mode. topsApp.py will not work for other Sentinel-1 acquisition formats such as Stripmap and ScanSAR. ISCE's stripmapApp.py supports interferometric stripmap processing of Sentinel-1 and other sensors. At this time, topsApp only supports SLC data from Sentinel-1 A and B. Processing is supported across the Sentinel-1 constellation, i.e. data acquired from A and B can be combined.

To illustrate the usage of topsApp.py, we will use a Sentinel-1 dataset capturing the surface deformation as result of the 12 November 2017 Mw7.3 earthquake that occurred in Iran.  The exercise runs the workflow step by step to generate a geocoded interferogram and offsets map. 

## 0. Initial setup of the notebook

The cell below performs initial setup of the notebook and must be run every time the notebook is used. It is possible to partially complete the exercise, close the notebook, and come back and continue later from that point, but this initialization must be re-run before restarting (as well as Step 1.2.1 if the slc download step 1.2 is skipped).  Initialization defines the processing locations as well as a few plotting routines that will be used throughout the tutorial, and this information is lost when the notebook is closed.

In [ ]:
from shutil import copyfile
from osgeo import gdal            ## GDAL support for reading virtual files
import os                         ## To create and remove directories
import matplotlib.pyplot as plt   ## For plotting
import numpy as np                ## Matrix calculations
import glob                       ## Retrieving list of files


## NOTE: When retrieiving the TOPS notebook from a different location,
## it is assumed that you have put it in your home dir under /work/notebooks/TOPS

## Defining the home and data directories at the processing location
home_dir = os.getenv("HOME")
tutorial_home_dir = os.path.abspath(os.path.join(home_dir, "work/notebooks/TOPS"))
slc_dir = os.path.join(tutorial_home_dir,'slc')
processing_DEM_dir  =  os.path.join(tutorial_home_dir,'DEM/DEM1')
geocoding_DEM_dir  =  os.path.join(tutorial_home_dir,'DEM/DEM3')
print("home directory: ", tutorial_home_dir)

# generate all the folders in case they do not exist yet
if not os.path.exists(tutorial_home_dir):
    os.makedirs(tutorial_home_dir)
if not os.path.exists(slc_dir):
    os.makedirs(slc_dir)
if not os.path.exists(processing_DEM_dir):
    os.makedirs(processing_DEM_dir)
if not os.path.exists(geocoding_DEM_dir):
    os.makedirs(geocoding_DEM_dir)
os.chdir(tutorial_home_dir)


# defining backup dirs in case of download issues on the local server
data_backup_dir = os.path.abspath(os.path.join(home_dir,"course-material/datasets/TOPS"))
slc_backup_dir = os.path.join(data_backup_dir,'slc')
processing_DEM_backup_dir  =  os.path.join(data_backup_dir,'DEM/DEM1')
geocoding_DEM_backup_dir  =  os.path.join(data_backup_dir,'DEM/DEM3')


def plotdata(GDALfilename,band=1,title=None,colormap='gray',aspect=1, datamin=None, datamax=None,draw_colorbar=True,colorbar_orientation="horizontal",background=None):
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    data = ds.GetRasterBand(band).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    
    # getting the min max of the axes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+data.shape[1]*deltax
    lasty = firsty+data.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])

    # put all zero values to nan and do not plot nan
    if background is None:
        try:
            data[data==0]=np.nan
        except:
            pass
    
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111)
    cax = ax.imshow(data, vmin = datamin, vmax=datamax, cmap=colormap,extent=[xmin,xmax,ymin,ymax])
    ax.set_title(title)
    if draw_colorbar is not None:
        cbar = fig.colorbar(cax,orientation=colorbar_orientation)
    ax.set_aspect(aspect)    
    plt.show()
    
    # clearing the data
    data = None
    
def plotcomplexdata(GDALfilename,title=None,aspect=1,datamin=None, datamax=None,draw_colorbar=None,colorbar_orientation="horizontal"):
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    slc = ds.GetRasterBand(1).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    
    # getting the min max of the axes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+slc.shape[1]*deltax
    lasty = firsty+slc.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])

    # put all zero values to nan and do not plot nan
    try:
        slc[slc==0]=np.nan
    except:
        pass

    
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(1,2,1)
    cax1=ax.imshow(np.abs(slc),vmin = datamin, vmax=datamax, cmap='gray',extent=[xmin,xmax,ymin,ymax])
    ax.set_title(title + " (amplitude)")
    if draw_colorbar is not None:
        cbar1 = fig.colorbar(cax1,orientation=colorbar_orientation)
    ax.set_aspect(aspect)

    ax = fig.add_subplot(1,2,2)
    cax2 =ax.imshow(np.angle(slc),cmap='rainbow',extent=[xmin,xmax,ymin,ymax])
    ax.set_title(title + " (phase [rad])")
    if draw_colorbar is not None:
        cbar2 = fig.colorbar(cax2,orientation=colorbar_orientation)
    ax.set_aspect(aspect)
    plt.show()
    
    # clearing the data
    slc = None
    
def plotstackdata(GDALfilename_wildcard,band=1,title=None,colormap='gray',aspect=1, datamin=None, datamax=None,draw_colorbar=True,colorbar_orientation="horizontal"):
    # get a list of all files matching the filename wildcard criteria
    GDALfilenames = glob.glob(os.path.abspath(GDALfilename_wildcard))
    
    # initialize empty numpy array
    for GDALfilename in GDALfilenames:
        ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
        data_temp = ds.GetRasterBand(band).ReadAsArray()   
        ds = None
        
        try:
            data
        except NameError:
            data = data_temp
        else:
            data = np.vstack((data,data_temp))

    # put all zero values to nan and do not plot nan
    try:
        data[data==0]=np.nan
    except:
        pass            
            
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111)
    cax = ax.imshow(data, vmin = datamin, vmax=datamax, cmap=colormap)
    ax.set_title(title)
    if draw_colorbar is not None:
        cbar = fig.colorbar(cax,orientation=colorbar_orientation)
    ax.set_aspect(aspect)    
    plt.show() 

    # clearing the data
    data = None
    
def plotstackcomplexdata(GDALfilename_wildcard,title=None,aspect=1, datamin=None, datamax=None,draw_colorbar=True,colorbar_orientation="horizontal"):
    # get a list of all files matching the filename wildcard criteria
    GDALfilenames = glob.glob(os.path.abspath(GDALfilename_wildcard))
    
    # initialize empty numpy array
    for GDALfilename in GDALfilenames:
        ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
        data_temp = ds.GetRasterBand(1).ReadAsArray()
        ds = None
        
        try:
            data
        except NameError:
            data = data_temp
        else:
            data = np.vstack((data,data_temp))

    # put all zero values to nan and do not plot nan
    try:
        data[data==0]=np.nan
    except:
        pass              
            
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(1,2,1)
    cax1=ax.imshow(np.abs(data),vmin = datamin, vmax=datamax, cmap='gray')
    ax.set_title(title + " (amplitude)")
    if draw_colorbar is not None:
        cbar1 = fig.colorbar(cax1,orientation=colorbar_orientation)
    ax.set_aspect(aspect)

    ax = fig.add_subplot(1,2,2)
    cax2 =ax.imshow(np.angle(data),cmap='rainbow')
    ax.set_title(title + " (phase [rad])")
    if draw_colorbar is not None:
        cbar2 = fig.colorbar(cax2,orientation=colorbar_orientation)
    ax.set_aspect(aspect)
    plt.show() 
    
    # clearing the data
    data = None

## 1. Overview of the tutorial input dataset

Let us first take a look at the dataset. For our dataset we are focusing on Ascending track 72 and acquisitions dates of 20171112 and 20171118. As the earthquake occurs along the frame boundaries we will use multiple Sentinel-1 SLC products for each acquisition.

![title](support_docs/region.png)

### 1.1 Background on TOPS mode

The TOPS acquisition strategy is different than conventional stripmap mode. TOPS stands for Terrain Observation with Progressive Scans. As the name indicates, the radar sensor performs a scan of the surface by electronically steering the antenna beam from a backward-pointing along-track direction to a forward-pointing along-track direction for a fixed range swath (also called a subswath).
![title](support_docs/tops_mode.png)
After a successful scan at that range extent, the antenna beam is electronically rolled back to its initial position, the range swath is electronically directed outward to a new area to increase coverage, and the next scan is made from backward to forward along track at this new range swath. After a third scan at a third range swath, the entire process is repeated over and over to create a continuous image as the satellite flies along.  The timing of the scans is such that there is a small geographic overlap between successive scans at a given range to ensure continuous coverage. Each scan at a given range swath is known as a "burst".  When inspecting one of the downloaded SLC products you will note that data are provided in 3 individual subswaths (IW1, IW2, IW3) each with a set of bursts. All together they form a Sentinel-1 frame, typically ~250 x 250 km in size. The bursts are approximately 20 km in length and overlap by 2 km. Due to the TOPS acquisition mode, the overlap region in successive bursts is seen from two different directions (forward looking and backward looking).
![title](support_docs/tops.png)
The ESA website provides detailed background information on Sentinel-1 products and technical information of the TOPS sensors. Whenever during the practical you are waiting for the processing to complete you could explore their webpage at https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar

### 1.2 SLC download

The ASF vertex page (https://www.asf.alaska.edu/sentinel/) and the SSARA GUI (http://web-services.unavco.org/brokered/ssara/gui) both offer a GUI to visually search for available Sentinel-1 data over your area of interest. Once you have found your data, you can download it from the GUI. ASF provides a bulk-download python script.

Alternatively, you can use SSARA from command line.

In [ ]:
!git clone https://github.com/bakerunavco/SSARA.git  

Running the client without any options will print the help message and describe the options.  Some usage examples are given in the help as well. 

In [ ]:
!SSARA/ssara_federated_query.py  

Take some time to perform some example searches in a terminal window to get a feel from how the client works.
<br>
<div class="alert alert-warning">
<b>TIP:</b> 
In case you are not sure about some of the SSARA API fields (e.g. the platform name, the polygon etc.), you can experiment with the SSARA GUI first. One potential solution is to draw a polygon without specifying much additional information, see what the search table returns for keywords, and use those on the command line API. 
</div>

The following command provides the necessary filters for the search and returns the 4 scenes we need for processing:

In [ ]:
!SSARA/ssara_federated_query.py --platform=sentinel-1B,sentinel-1A --relativeOrbit=72 -s 2017-11-11 -e 2017-11-18 --intersectsWith='POLYGON((45 35.25,45 34,46.5 34,46.5 35.25,45 35.25))' --print 

By changing the **--print** to **--download** you can download the files once you set up your password and username for SSARA in the **password_config.py** file.

Below we include a small python script to download the data which allows you to directly parse your pasword and username at the top of the function (this will take a few minutes for each file).

In [ ]:
ASF_USER = ""
ASF_PASS = ""

files = ['https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20171111T150004_20171111T150032_019219_0208AF_EE89.zip',
         'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20171111T150029_20171111T150056_019219_0208AF_BF55.zip',
         'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20171117T145900_20171117T145928_008323_00EBAB_B716.zip',
         'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20171117T145926_20171117T145953_008323_00EBAB_AFB8.zip']

if len(ASF_USER)==0 or len(ASF_PASS)==0:
    raise Exception("Specifiy your ASF password and user (earthdata log-in)")
    
for file in files:
    filename = os.path.basename(file)
    
    if not os.path.exists(os.path.join(slc_dir,filename)):
        cmd = "wget {0} --user={1} --password={2}".format(file,ASF_USER, ASF_PASS)
        print(cmd)
        os.chdir(slc_dir)
        os.system(cmd)

    else:
        print(filename + " already exists")
    os.chdir(tutorial_home_dir)

Files size are around 4.3GB

In [ ]:
ls -lh slc

#### Download issues

In case the download does not work, we have pre-downloaded the required files. Run the routine below to copy these files into your work directory.

In [ ]:
files = ['https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20171111T150004_20171111T150032_019219_0208AF_EE89.zip',
         'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20171111T150029_20171111T150056_019219_0208AF_BF55.zip',
         'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20171117T145900_20171117T145928_008323_00EBAB_B716.zip',
         'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20171117T145926_20171117T145953_008323_00EBAB_AFB8.zip']

for file in files:
    filename = os.path.basename(file)    
    if not os.path.exists(os.path.join(slc_dir,filename)):  
        cmd = 'cp -r ' + os.path.join(slc_backup_dir,filename) + " " + slc_dir
        os.system(cmd)
        print(filename + " done")
    else:
        print(filename + " already exists")
    os.chdir(tutorial_home_dir)
    

Files size are around 4.3GB

### 1.2.1 Set/Reset to home directory

This step is required each time the notebook is restarted and the slc download is skipped.  It can also be run in the initial run, though it is not needed since the directory is reset during the slc download process.

In [ ]:
os.chdir(tutorial_home_dir)

In [ ]:
!ls -lh slc

### 1.3 SLC filenaming convention

TOPS SLC product files delivered from ESA are zip archives. When unpacked the zip extension will be replaced by SAFE. The products are therefore also frequently called SAFE files. topsApp.py can read the data from either a zip file or a SAFE file. To limit disk usage, it is recommended to not unzip the individual files.

The zip or SAFE filenames provide information on the product type, the polarization, and the start and stop acquisition time. For example: S1A_IW_SLC__1SDV_20171111T150004_20171111T150032_019219_0208AF_EE89.zip 
- Type = slc
- Polarization = Dual pole 
- Date = 20171111
- UTC time of acquisition = ~15:00
- Sensing start for the acquisition was 20171111 at 15:00:04


### 1.3 Orbits and Instrument file download

In addition to the **SAFE files**, **orbit files** and the **auxiliary instrument files** are required for ISCE processing. Both the orbit and instrument files are provided by ESA and can be downloaded at: https://qc.sentinel1.eo.esa.int/.

We have pre-downloaded the precise orbit files as well as the instrument files for our test dataset. Run the routine below to copy these files into the aux folder.

In [ ]:
aux_dir = os.path.join(tutorial_home_dir,'aux')

files = ['aux_cal/S1A_AUX_CAL_V20140908T000000_G20140909T130257.SAFE',
        'aux_cal/S1A_AUX_CAL_V20140915T100000_G20151125T103928.SAFE',
        'aux_poeorb/S1A_OPER_AUX_POEORB_OPOD_20171201T121227_V20171110T225942_20171112T005942.EOF',
        'aux_poeorb/S1B_OPER_AUX_POEORB_OPOD_20171207T111223_V20171116T225942_20171118T005942.EOF',]

for file in files:
    if not os.path.exists(os.path.join(aux_dir,file)):        
        cmd = "cp -r " + os.path.join(data_backup_dir,'aux',file) + " " + os.path.join(aux_dir,file)
        os.system(cmd)
        print(file + " done")

    else:
        print(file + " already exists")
    os.chdir(tutorial_home_dir)

In [ ]:
ls aux/*

Although Sentinel-1 restituted orbits (RESORB) are of good quality, it is recommended to use the precise orbits (POEORB) when available. Typically, precise orbits are available with a 15 to 20-day lag from the day of the acquisition.

<div class="alert alert-danger">
<b>AUX_CAL:</b> 
AUX_CAL information is used for **antenna pattern correction** of SAFE products with **IPF verison 002.36**. If all your SAFE products are from another IPF version, then no AUX files are needed.
</div>


## 2. topsApp.py input variables

Like the other apps in ISCE, the input variables to topsApp.py are controlled through an app xml file. All apps in ISCE have example xml files included in the ISCE distribution. You can find these under **/examples/input_files**. For convenience, we have included the *topsApp.xml* and *master_TOPS_SENTINEL1.xml* example in the support_docs folder. 

### 2.1 Required versus optional topsApp.py inputs

The example *topsApp.xml* contains all input variables with a description. Let us first read this file. You can open the file by launching a **terminal** and using your preferred editor to open the file. For vim type:
```
    vim support_docs/example/topsApp.xml
    vim support_docs/example/master_TOPS_SENTINEL1.xml
```
When it comes to the actual processing with topsApp.py, you do not need to specify all the input variables as shown in the example topsApp.xml. Defaults will be assumed when properties are not set by the user. You can get a simple table overview of the required variables by calling the help of topsApp.py.

In [ ]:
!topsApp.py --help

From the table, you can see that the master and slave components are to be specified. This can be done directly with its specific properties in the topsApp.xml, or alternatively, one can point to a dedicated xml file for the master and slave, each of which contain their individual properties. The required properties for the master and slave should at least include orbit information as "orbit directory", the auxiliary instrument information as "auxiliary data directory", and a list of products under the "safe" property tag. 

<br>
<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
If you specify a **region of interest**, make sure it covers at last two bursts. If this is not the case, ESD will fail in the processing as it is being estimated from the burst overlap region. You could always decide to only geocode a smaller region with the ** ** property.
</div>

### 2.2 topsApp inputs for this tutorial

Let us now examine the files *topsApp.xml*, *master.xml* and *slave.xml*  that will be used for this tutorial.

#### topsApp.xml

```xml
<?xml version="1.0" encoding="UTF-8"?>
<topsApp>
  <component name="topsinsar">
    <property name="Sensor name">SENTINEL1</property>
    <component name="master">
        <catalog>master.xml</catalog>
    </component>
    <component name="slave">
        <catalog>slave.xml</catalog>
    </component>
    <property name="swaths">[3]</property>
    <property name="range looks">19</property>
    <property name="azimuth looks">7</property>
    <property name="region of interest">[34, 35.25 , 44.75, 46.5]</property>
    <!--<property name="geocode demfilename">../../DEM/DEM3/Coarse_demLat_N33_N36_Lon_E045_E047.dem.wgs84</property>-->
    <!--<property name="demFilename">../../DEM/DEM1/demLat_N33_N36_Lon_E045_E047.dem.wgs84</property>-->
    <property name="do unwrap">True</property>
    <property name="unwrapper name">snaphu_mcf</property>
    <!--<property name="do unwrap2stage">True</property>-->
    <property name="do denseoffsets">True</property>
    <!--property name="geocode list">['merged/phsig.cor', 'merged/filt_topophase.unw', 'merged/los.rdr', 'merged/topophase.flat', 'merged/filt_topophase.flat','merged/topophase.cor','merged/filt_topophase.unw.conncomp']</property>-->
  </component>
</topsApp>
```


- The master and slave components refer  to their own *.xml* files 
- The **swaths** property controls the number of swaths to be processed. As the earthquake occurred in the subswath three, we can directly limit the list to the single entry list [3] only.
- We specify the **range looks** and **azimuth looks** to be 19 and 7. The range resolution for sentinel varies from near to far range, but is roughly 5m, while the azimuth resolution is approximately 15m, leading to a multi-looked product that will be approximately 95m by 105m.
- By specifying the **region of interest** as [S, N, W, E] to only capture the extent of the earthquake, topsApp.py will only extract those bursts from subswath 3 needed to cover the earthquake.
- By default, topsApp can download a DEM on the fly. By including **demFilename** a local DEM can be specified as input for the processing. For this notebook exercise, a sample 1-arc second DEM in WGS84 is provided in the DEM/DEM1 folder.
- By default, the geocoding in topsApp.py is performed at the same sampling as processing DEM. However, a different DEM *to be used specifically for geocoding* can be specified using the **geocode demfilename** property. In this notebook, we provide a DEM in WGS84 sampled at 3-arc seconds to be used specifically for geocoding. It is stored in the DEM/DEM3 folder.  
- By default, no unwrapping is done. In order to turn it on, set the property **do unwrap** to *True*.
- In case unwrapping is requested, the default unwrapping strategy to be applied is the *icu* unwrapping method. For this tutorial, we will use *snaphu_mcf*.
- Lastly, we request topsApp.py to run the dense-offsets using the **do denseoffsets** property. By enabling this, topsApp.py will estimate the range and azimuth offsets on the amplitude of the master and slave SLC files.


You will see that a few of the above properties are commented out in the xml files provided. We will come back to these later in the tutorial.  The commented properties have the form:
```xml
<!--<property> ... </property>--> 
```

#### master.xml and slave.xml

**master.xml**
``` xml
<component name="master">
    <property name="orbit directory">../../aux/aux_poeorb</property>
    <property name="auxiliary data directory">../../aux/aux_cal</property>
    <property name="output directory">master</property>
    <property name="safe">['../../slc/S1B_IW_SLC__1SDV_20171117T145900_20171117T145928_008323_00EBAB_B716.zip','../../slc/S1B_IW_SLC__1SDV_20171117T145926_20171117T145953_008323_00EBAB_AFB8.zip']</property>
</component>
```

**slave.xml**
``` xml
<component name="slave">
    <property name="orbit directory">../../aux/aux_poeorb</property>
    <property name="auxiliary data directory">../../aux/aux_cal</property>
    <property name="output directory">slave</property>
    <property name="safe">['../../slc/S1A_IW_SLC__1SDV_20171111T150004_20171111T150032_019219_0208AF_EE89.zip']</property>
</component>
```

- The value associated with the master **safe** property corresponds to a list of SAFE files acquired on 20171117. For the slave only, one SAFE files is included, acquired on 20171111. The paths to each SAFE file points to the slc directory.
- The **orbit directory** and **auxiliary data directory** point respectively to the directory where we have stored the POEORB (precise) orbits for the example, and the Auxiliary instrument files for Sentinel-1.


<br>
<div class="alert alert-warning">
<b>SIGN CONVENTION:</b> 
By selecting the master to be acquired after the slave, and keeping in mind that the interferogram formation is master* conj(slave), then a positive phase value for the interferogram indicates the surface has moved away from the satellite between 20171111 and 20171117. 
</div>


## 3. topsApp.py processing steps

The topsApp.py workflow can be called with a single command-line call to topsApp.py; by default it will run all the required processing steps with inputs pulled from the topsApp.xml file. Although this is an attractive feature, it is recommended to run topsApp.py with “steps” enabled. This will allow you to re-start the processing from a given processing step. If “steps” are not used, users must restart processing from the beginning of the workflow after fixing any downstream issues with the processing. 

The "--help" switch lists all the steps involved in the processing:

In [ ]:
!topsApp.py --help --steps

<br>
<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
**Steps are to be run in the following prescribed order**:
*<center>
['startup', 'preprocess', 'computeBaselines', 'verifyDEM', 'topo']
['subsetoverlaps', 'coarseoffsets', 'coarseresamp', 'overlapifg', 'prepesd']
['esd', 'rangecoreg', 'fineoffsets', 'fineresamp', 'burstifg']
['mergebursts', 'filter', 'unwrap', 'unwrap2stage', 'geocode']
['denseoffsets', 'filteroffsets', 'geocodeoffsets']
</center>*
</div>


### 3.0 Directory setup

Before we begin step by step processing with topsApp.py, we first setup the processing directory in the insar folder as **masteryyyymmdd_slaveyyyymmdd**, and copy the template topsApp.xml file (*ls support_docs/insar/topsApp.xml*) to the processing location.

In [ ]:
import os
from shutil import copyfile

## Directory where we will perform the topsApp processing
processing_dir  =  os.path.join(tutorial_home_dir,'insar/20171117_20171111')

## Template xml for this tutorial
topsAppXml_original =  os.path.join(tutorial_home_dir,'support_docs/insar/topsApp.xml')  
masterXml_original =  os.path.join(tutorial_home_dir,'support_docs/insar/master.xml')  
slaveXml_original =  os.path.join(tutorial_home_dir,'support_docs/insar/slave.xml')  



## Check if output directory already exists. If not create it
if not os.path.isdir(processing_dir):
    os.makedirs(processing_dir)

## Check if the topsApp.xml file already exists, if not copy the example for the excerisize
if not os.path.isfile(os.path.join(processing_dir,'topsApp.xml')):
    copyfile(topsAppXml_original,os.path.join(processing_dir,'topsApp.xml'))
else:
    print(os.path.join(processing_dir,'topsApp.xml') + " already exist, will not overwrite")
if not os.path.isfile(os.path.join(processing_dir,'master.xml')):
    copyfile(masterXml_original,os.path.join(processing_dir,'master.xml'))
else:
    print(os.path.join(processing_dir,'master.xml') + " already exist, will not overwrite")
if not os.path.isfile(os.path.join(processing_dir,'slave.xml')):
    copyfile(slaveXml_original,os.path.join(processing_dir,'slave.xml'))
else:
    print(os.path.join(processing_dir,'slave.xml') + " already exist, will not overwrite")

We will now "cd" into this directory where the processing will be done.

In [ ]:
os.chdir(processing_dir)

### 3.1 Step Startup

We will start with running the first step **startup**. 

In [ ]:
!topsApp.py --dostep=startup

When topsApp.py is run in steps, PICKLE files are used to store state information between the steps.  A PICKLE folder is created during the startup step. The PICKLE folder is used to store the processing parameters for each processing step. By exploring the pickle folder, you will find a binary pickle file and an *xml* file associated with the **startup** step.

In [ ]:
ls PICKLE

The information contained within the **startup** pickle and *xml* files allows topsApp to start or re-start processing from where **startup** was completed. 

### 3.2 Step Preprocess

Keeping in mind that the order of steps matter, we move to the second step of topsApp.py processing, which is to **preprocess** the data.

In [ ]:
!topsApp.py --dostep=preprocess

During preprocessing, the orbits, the IPF (Instrument Processing Facility, which is the processing version used by ESA to focus the data), the bursts, and, if needed, the antenna pattern are extracted. 

Note if you had processed the complete region without limitation, there would be three subswaths (IW1,IW2,IW3). For our example,
we limited the processing to IW3 alone.  Therefore, for our tutorial, within the slave and master folder you will find only an **IW3** folder and an **IW3.xml** file.

In [ ]:
ls master/*

The **IW3.xml** file contains metadata information specific for each subswath (e.g. orbit state vectors, doppler, sensing times, IPF version). You can open the file using your preferred editor from the command line:
```
    vim master/IW3.xml
```
The **IW3** folder contains the unpacked bursts and their meta-data.
Typically you will find *.xml* and *.vrt* files, and in certain cases *.slc* files, as described below.
-	If a bounding box is specified, only the master bursts covering the box are unpacked, otherwise the complete master is used. The bursts of the master in master/IW* are relabeled from 1 to n.
-	The slave/IW* folders typically contain a larger set of bursts to cover the master extent completely, where the bursts are labelled from 1 to m.  The burst numbering is not coordinated between master and slave folders: burst 1 in the master does not necessarily correspond to burst 1 in the slave.
-	All data are unpacked virtually (i.e., only *.xml* and *.vrt* files in *IW* folders) unless: (1) user requested to physically unpack the data, which can be requested in topsApp.xml by property **usevirtualfiles**, or (2) if an antenna pattern correction needs to be applied (not controlled by user) which happens mainly for the initially acquired Sentinel-1 data.
-	The IPF version of unpacked bursts is tracked and a combined IFP is assigned for the master and one for the slave. Note: bursts with varying IPF cannot be stitched and attempting to do so will cause an error message. 

<br>
<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
There is a gap in spatial coverage, e.g. you are missing the middle SAFE file.
</div>
<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
The SAFE frames that need to be stitched do not have a consistent IPF version
</div>

### 3.3 Step computeBaselines

In [ ]:
!topsApp.py --dostep=computeBaselines

In this step, the perpendicular and parallel baselines are computed using the orbit (state vector information) for the first and last burst of the master image (center range). Each subswath is processed individually, with output sent to the screen. As the processing for the tutorial is limited to IW3, the baselines are only computed for this subswath.

### 3.4 Step verifyDEM

In [ ]:
!topsApp.py --dostep=verifyDEM

This step will check the DEM file specified in the topsApp.xml. If no DEM file has been specified, topsApp.py will download the DEM on the fly and track the filename for subsequent processing. 

<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
You did not set your earthdata credentials as detailed in the ISCE installation.
</div>
<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
The DEM ftp site is down and returns no data tiles.
</div>


#### Directly move to 3.5 if verifyDEM does not fail:

If your DEM download failed and you are unable to resolve the issue, you can use the DEM provided with this tutorial. This requires three steps to complete, before you can rerun the **verifyDEM** step:

1) Copy over the DEM from the backup folder to your processing location

In [ ]:
cmd = "cp -r " + os.path.join(processing_DEM_backup_dir,'*') + " " + processing_DEM_dir
print(cmd)
os.system(cmd)
print('done')

2) To use this DEM, you will need to edit the **topsApp.xml** file and uncomment the property for the **demFilename**.
``` vim
    vim insar/20171117_20171111/topsApp.xml
```
Remember the *xml* guidelines:
```xml
<!--<property> ..COMMENTED.. </property>--> 
<property> ..UNCOMMENTED.. </property>
```

3) Update the DEM metadata information. In the processing the DEM metadata such as the DEM path is read from the *xml* information. As the DEM was copied from another machine, we need to update the path. **fixImageXml.py** allows you to do this from the command line.

In [ ]:
!fixImageXml.py -h

From the help, you can see that the **-f** option will update the path in the xml file with an absolute path. Now let us run the command:

In [ ]:
os.chdir(processing_DEM_dir)
!fixImageXml.py -i demLat_N33_N36_Lon_E045_E047.dem.wgs84 -f
os.chdir(processing_dir)

Now rerun the **verifyDEM** step to ensure the new DEM information is correctly loaded into the processing:

In [ ]:
!topsApp.py --dostep=verifyDEM

### 3.5 Step topo

During topo processing, the DEM is mapped into the radar coordinates of the master. As output this generates the **master_geom** folder containing the longitude (*lon_XX.rdr*), latitude (*lat_XX.rdr*), height (*hgt_XX.rdr*) and LOS angles (*los_XX.rdr*) on a pixel by pixel grid for each burst (*XX*). This step is the most time-consuming step. It is parallelized for performance and can also be ran with GPU support enabled. 

For our tutorial, anticipate this step will take 20+ min in CPU mode with 4 treads. 
***Depending on the size of the class the instructor might recommend to decrease the number of treads, pair up in teams, or stagger this processing step***.

This is a good opportunity to familiarize yourself a bit more with the TOPS mode and input parameters.
- Can you find which property in the topsApp.xml controls the GPU processing? Tip: see “topsApp.py --help”.
- Can you find the typical incidence angle range for IW3? Tip: You could try to load the los.rdr file for a burst or search the ESA website for TOPS documentation.

Once the step is complete you can have a look at the files that have been generated for each burst:

In [ ]:
!export OMP_NUM_TREADS=4; topsApp.py --dostep=topo

In [ ]:
ls geom_master/*

Were you able to figure out the incidence angle for subswath 3? It is about 41$^\circ$-46$^\circ$. This infomation is contained in the *los.XX.rdr* files.  Its first band contains the incidence angle and the second band the azimuth angle of the satellite. There is an easy way to retrieve its average value with GDAL:

In [ ]:
!gdalinfo -stats geom_master/IW3/los_07.rdr.vrt

In [ ]:
plotdata('geom_master/IW3/hgt_01.rdr',1,'IW3: Height of Burst 1 [meter]','terrain')
plotdata('geom_master/IW3/los_01.rdr',1,'IW3: Incidence angle of Burst 1 [degrees]','jet')
plotdata('geom_master/IW3/los_01.rdr',2,'IW3: Azimuth angle of Burst 1 [degrees]','jet')

### 3.6 Step subsetoverlaps

Due to the large doppler frequency variation inherent to the TOPS acquisition mode, images supporting interferometry must be coregistered in the azimuth direction to better than 0.001 pixels, compared to regular stripmap data (0.1 of a pixel). While conventional cross-correlation of the amplitude works for the range direction, it does not provide sufficient accuracy for azimuth. The solution is to apply an Enhanced Spectral Diversity (ESD) approach to estimate the azimuth coregistration. In ESD processing, a double difference interferogram is made between the master and slave in the burst overlap region. As the range displacement is cancelled out, this interferogram will only show azimuthal motion. In absence of large ground deformation this azimuthal motion can be interpreted as an azimuthal coregistration offset. 

The following processing steps up to ESD are specific to burst overlap regions alone. 

By running the **subsetoverlaps** step, the top and bottom overlap between bursts is computed for the master geometry. The information is then stored within the *overlaps* folder of the master directory.  

Note, the nomenclature "top" and "bottom" can be confusing.  In this case, it is **not** referring to the top and bottom of the burst. It is referring to burst *n* as "top" and burst *n+1* as "bottom", visualizing it as the earlier image being laid on top of the later image, which is on the bottom. 

![title](support_docs/esd.png)
(Figure from Fattahi et. al., SCEC)

In [ ]:
!topsApp.py --dostep=subsetoverlaps

In [ ]:
ls  master/overlaps/*

For each subswath being processed, you will find a *bottom_IW[].xml* and a *top_IW[].xml* file, associated with the overlap region in the the bottom and top overlapping bursts, respectively. Like before, only a IW3 folder is present for this tutorial. If you explore the file, you will find  additional information such as the FM rate and the doppler information.
```
    vim master/overlaps/bottom_IW3.xml
```
Within the *overlaps* directory you will also find for each processed subswath a folder containing the cropped SLC's for each burst overlap region. The convention for top and bottom is as follows:
- Burst_bot_01_02.slc and Burst_top_01_02.slc both refer to the same burst overlap region, where:
    - Burst_bot_01_02.slc is the part of burst 2 (bot burst)
    - Burst_top_01_02.slc is the part of burst 1 (top burst)
- Burst_bot_02_03.slc and Burst_top_02_03.slc both refer to the same burst overlap region, where:
    - Burst_bot_02_03.slc is the part of burst 3
    - Burst_top_02_03.slc is the part of burst 2
- etc...

Though the overlap is not large, we will try to visualize the overlap by comparing:
- Burst_bot_01_02.slc and Burst_top_01_02.slc

Can you spot the overlap with respect to the full burst?

In [ ]:
plotcomplexdata('master/overlaps/IW3/burst_bot_01_02.slc.vrt','bot_01_02 (from Burst 2)',aspect=10,datamin=0,datamax=150)
plotcomplexdata('master/overlaps/IW3/burst_top_01_02.slc.vrt','top_01_02 (from Burst 1)',aspect=10,datamin=0,datamax=150)
plotcomplexdata('master/IW3/burst_01.slc.vrt','Burst 1',aspect=10,datamin=0,datamax=150)
plotcomplexdata('master/IW3/burst_02.slc.vrt','Burst 2',aspect=10,datamin=0,datamax=150,draw_colorbar=True,colorbar_orientation="horizontal")

### 3.7 Step coarseoffsets

A coarse coregistration can be done using the orbits alone. This is also referred to as geometric coregistration. As ESD is applied in burst overlap region, there is no need to apply this **coarseoffsets** estimation step to the complete SLC. Instead, we estimate the coarse offset for each burst overlap as identified in the previous step.

In [ ]:
!topsApp.py --dostep=coarseoffsets

As output this generates a "*coarse_offsets/overlaps*" folder, within which is a subswath breakdown (*IW*) containing pixel-by-pixel azimuth offsets (*azimuth_[top,bot]_XX_YY.off*) and range offsets (*range_[top,bot]_XX_YY.off*) for each burst overlap.

In [ ]:
ls coarse_offsets/overlaps/IW3/*

Let us plot the average azimuth offset is for burst overlap: bot_01_02

In [ ]:
plotdata('coarse_offsets/overlaps/IW3/azimuth_bot_01_02.off.vrt',1,'Azimuth bot_01_02','gray',aspect=10)
!gdalinfo -stats coarse_offsets/overlaps/IW3/azimuth_bot_01_02.off.vrt

Now do the same for the range offset.

In [ ]:
plotdata('coarse_offsets/overlaps/IW3/range_bot_01_02.off.vrt',1,'Range bot_01_02','gray',aspect=10)
!gdalinfo -stats coarse_offsets/overlaps/IW3/range_bot_01_02.off.vrt


### 3.8 Step coarseresamp

Both master and slave need to be in a common geometry prior to interferogram formation. In this step we will be taking the outputs from the coarse offset estimation and resample the slave burst overlaps into the master geometry.

In [ ]:
!topsApp.py --dostep=coarseresamp

The output of this step is stored in the "*coarse_coreg/overlaps/*" folder for each subswath. 

In [ ]:
ls coarse_coreg/overlaps/IW3/*

### 3.9 Step overlapifg

Now that the master and slave burst overlap SLC's are in the same geometry we can compute the interferogram between the master and the slave. As there is a bottom and top set of SLC's, there will be two interferograms for each burst overlap: 
$$\Delta\Phi_{\text{bot}}  = \Phi_{\text{bot}}^{\text{master}}*conj\left( \Phi_{\text{bot}}^{\text{slave}}\right) $$
$$\Delta\Phi_{\text{top}}  = \Phi_{\text{top}}^{\text{master}}*conj\left( \Phi_{\text{top}}^{\text{slave}}\right) $$

In [ ]:
!topsApp.py --dostep=overlapifg

As output this generates a "*coarse_interferogram/overlaps*" folder, with in it a subswath breakdown (*IW*) containing the inteferograms (*burst_[top,bot]_XX_YY.int*) and multi-looked inteferograms (*burst_[top,bot]_XX_YY.7alks_19rlks.int*) for each burst overlap.

In [ ]:
ls coarse_interferogram/overlaps/IW3*

Let us plot all the multi-looked interferograms for the "top burst" overlaps together.

In [ ]:
plotstackcomplexdata('coarse_interferogram/overlaps/IW3/burst_top_*7alks_19rlks.int',title="Top overlaps ",aspect=10, datamin=0, datamax=6000,draw_colorbar=True,colorbar_orientation="horizontal")

As you can see there is a large fringe-rate in the interferograms, especially towards the center of our study area. Do you have any thoughts why this might be?

### 3.10 Step prepesd

Now that the bottom and top interferograms have been calculated the double difference interferogram can be calculated, this procedure is also referred to as spectral diversity: 
$$\Delta\Phi_{\text{ESD}} = \Delta\Phi_{\text{bot}}*conj\left(\Delta\Phi_{\text{top}}\right)$$

In [ ]:
!topsApp.py --dostep=prepesd

As output this generates a "*ESD*" folder, with in it the inteferograms (*overlap_[IW?]_XX.int*) and multi-looked interferograms (*overlap_[IW?]_XX.5alks_15rlks.int*) for each burst overlap *XX*.

### 3.11 Step esd

The double difference interferogram only captures displacements in the azimuth direction. During the **esd** step, the phase of the double difference interferogram for each burst overlap is converted to pixel offsets using the azimuth frequency and difference of the Doppler Centroid frequency between the forward and backward-looking geometry. 

$$ \text{offset}_{\text{azimuth}}= \frac{f_{\text{azimuth}}}{2\pi} \frac{\text{arg}\left[e^{ j\Delta\Phi_{\text{ESD}}} \right]}{\Delta f_{\text{DC}}^{\text{overlap}}}$$

Once completed, all the offsets are combined together, and the mean of the distribution is taken as the azimuth coregistration offset. Note only those pixels with a coherence > than 0.85 are considered. 

<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
Coherence is too bad to find reliable point to estimate the ESD threshold. Either take one the following two options listed below.
</div>
<br>
<div class="alert alert-warning">
<b>ESD threshold:</b> 
The ESD coherence threshold can be changed by the user in the **topsApp.xml** using the property **ESD coherence threshold"**. It is recommended not to go below 0.7. 
</div>
<br>
<div class="alert alert-warning">
<b>SKIP ESD:</b> 
In case you do not want to perform ESD, e.g. the coherence is to bad in the overlap regions, you could opt to skip ESD and only use the orbit information. This is controlled via **topsApp.xml** with the property **do ESD**. 
</div>

In [ ]:
!topsApp.py --dostep=esd

At the completion of the **esd** step you will see that *combined_*\**[.cor,.int,freq.bin]* files as well as an *ESDmisregistration.jpg* figure have been added to the "*ESD*" folder.

In [ ]:
ls ESD/combined* ESD/ESDmisregistration*

The combined coherence and interferogram files are used as input to the ESD azimuth offset estimation:

In [ ]:
plotdata('ESD/combined_IW3.cor',1,title="coherence [-]",aspect=10, datamin=0, datamax=1,background='yes')
plotcomplexdata('ESD/combined_IW3.int',title="Diff IFG [-]",aspect=10,datamin=100000, datamax=100000000,draw_colorbar=True)


In general, high coherence can be observed (> 0.85), and thus most pixels will be used in the ESD histogram generation. The interferogram shows strong azimuthal motion at the location of the earthquake. Away from the earthquake the impact is less, showing on average ~0 rad of azimuthal phase. Let us investigate if this potentially has biased the ESD estimation by inspecting the historgram figure that was created as part of the **esd** step:

In [ ]:
import matplotlib.image as mpimg
img= mpimg.imread("ESD/ESDmisregistration.png")
fig = plt.figure(figsize=(10, 11))
plt.imshow(img);

<div class="alert alert-warning">
<b>TIP:</b> 
It is always good to inspect the inputs of the ESD calculation, as there are sources such as ionosphere and tectonic deformation that can cause an apparent azimuthal shift, potentially biasing the mean of the ESD estimation.
</div>

### 3.12 Step rangecoreg

In the next step we will estimate the offset in the range direction by doing an amplitude cross-correlation between the master and slave burst overlaps. As for the azimuth offsets, a histogram of the range offset is used to estimate the mean range offset for the complete SLC. Note only pixels with an SNR of 8 or above are considered.

In [ ]:
!topsApp.py --dostep=rangecoreg

Let us vizualize the results:

In [ ]:
ls ESD

In [ ]:
import matplotlib.image as mpimg
img= mpimg.imread("ESD/rangeMisregistration.jpg")
fig = plt.figure(figsize=(10, 11))
plt.imshow(img);

### 3.13 Step fineoffsets

Now that we have estimated the range and azimuth offsets using the burst overlap regions, we will apply them to the full bursts in the **fineoffsets** step. 

In [ ]:
!topsApp.py --dostep=fineoffsets

The output is stored in the *fine_offsets* folder for each subswath, with an azimuth offset file (*azimuth_XX.off*) and a range offset file (*range_XX.off*) for each burst *XX*.

In [ ]:
ls fine_offsets/*

### 3.14 Step fineresamp

The burst fine offset files just created are used to resample the slave burst SLC's into the master geometry using the **fineresamp** step.

In [ ]:
!topsApp.py --dostep=fineresamp

The output is stored in the *coreg_fine* folder for each subswath, containing the resampled slave burst SLC's (*burst_XX.slc*).

### 3.15 Step burstifg

The slave and master burst SLC's are now in a common geometry, and interferograms can be computed for each burst. We will do this as part of the **burstifg** step.

In [ ]:
!topsApp.py --dostep=burstifg

This step generates a "*fine_interferogram*" folder, within which is a subswath breakdown (*IW*), each containing their respective interferograms (*burst_XX.int*), the coherence (*burst_XX.cor*), and the multilooked interferograms (*burst_XX.7alks_19rlks.int*) for each burst *XX*.

In [ ]:
ls fine_interferogram/*

Here is a plot of one of the burst interferograms:

In [ ]:
plotcomplexdata('fine_interferogram/IW3/burst_04.7alks_19rlks.int.vrt',title="Burst 4 IFG ",aspect=3,datamin=0, datamax=10000,draw_colorbar=True)

### 3.16 Step mergebursts

As you can tell from the figure, it is hard to capture the full extent of the earthquake deformation from a single burst. In the **mergebursts** step, we will combine the different bursts together to generate a combined product.

In [ ]:
!topsApp.py --dostep=mergebursts

The output is written into the "*merged*" folder. This folder contains the merged interferogram (**topophase.flat**), the coherence (**topophase.cor**), the line-of-sight (LOS) angles (**los.rdr**), the heights (**z.rdr**), the longitude (**lon.rdr**), and the latitude information (**lat.rdr**). These files are multilooked, while files with a *.full* suffix refer to the full resolution data. You can also find a merged SLC for the master (**master.slc.full.vrt**) and the slave (**master.slc.full.vrt**). These files are generated if **do denseoffsets** property is set to *True* in **topsApp.xml**. You can use *gdalinfo* to explore the format of these files and the number of bands.


Here is a plot of the merged intergerogram:

In [ ]:
plotcomplexdata('merged/topophase.flat.vrt',title="MERGED IFG ",aspect=3,datamin=0, datamax=10000,draw_colorbar=True)

At first sight the stitching appears to be successful, with no obvious discontinuities in the earthquake region across bursts. A closer inspection shows that the two top burst edges have a slight misalignment. This is likely introduced by either the ionosphere or leakage of the earthquake azimuth deformation into the ESD estimation procedure. Two potential avenues can be further explored:
#1) In addition to the ESD threshold, mask out pixels within the earthquake region. i.e. let the ESD estimation be described by the far-field.
#2) Specify not to use ESD in the topsApp.xml. Instead only use the orbit information for coregistration purposes.

### 3.17 Step filter

Multilooking reduces noise, but it is often insufficient for interpreting the interferogram or unwrapping. We often apply additional filtering to improve the SNR.

In [ ]:
!topsApp.py --dostep=filter

The outputs of the filtering step are added to the merged folder, where the filtered interferogram is called **filt_topophase.flat**, and the filtered coherence **phsig.cor**. Note that the latter is different from conventional coherence: it is based on an estimate of the standard deviation of the phase translated back to coherence through the Cramer-Rao bound relationship:

$\gamma = \frac{1}{\sqrt{1+2N\sigma_\phi^2}}$

Let us compare the filtered and unfiltered interferograms:

In [ ]:
plotcomplexdata('merged/topophase.flat.vrt',title="MERGED IFG ",aspect=3,datamin=0, datamax=10000,draw_colorbar=True)
plotcomplexdata('merged/filt_topophase.flat.vrt',title="MERGED FILT IFG ",aspect=1,datamin=0, datamax=10000,draw_colorbar=True)

<div class="alert alert-warning">
<b>FILTER STRENGTH:</b> 
The amount of filtering can be controlled through the **"filter strength"** property in the **topsApp.xml file**.</div>
<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
As a user you can change the filtering strength. Care should be taken, as a smoother and cleaner interferogram does not necessarily imply a better interferogram. Always inspect the interferogram before and after filtering and assess if false signals are introduced or if dense fringes are oversmoothed due to overfiltering. Look carefully at noisy regions, where fringes from coherent areas "bleed" into noisy areas.
</div>


### 3.18 Step unwrap

In the next step we will use **snaphu** to unwrap the interferogram:

In [ ]:
!topsApp.py --dostep=unwrap

The unwrapped output is added to the "*merged*" folder. The unwrapped interferogram is contained in **filt_topophase.unw**. You will also find the connected components stored in **filt_topophase.conncomp**. We plot them below and inspect the results for potential errors.

<div class="alert alert-warning">
<b>INSPECT CONNECTED COMPONENT OUPUT:</b> 
The connected component file is a metric of the unwrapping performance. It provides information on which parts of the unwrapped image are connected. Regions with a different connected component can have a phase jump equivalent to a multiple of 2 $\pi$. It is therefore important to always inspect both the phase and the connected component output together. </div>

<div class="alert alert-warning">
<b>CONNECTED COMPONENT NUMBERS:</b> 
Connected component 0 refers to those pixels which cannot be reliably unwrapped. These should be masked out before doing any sort of modeling. The other connected components increment by 1 per isolated region.
</div>
<div class="alert alert-danger">
<b>UNWRAPPING ERROR EXAMPLE:</b> 
To see unwrapped result with non-physical phase jumps, run the "icu" unwrapper by changing the **unwrapper name** property in **topsApp.xml** to *icu*. Note that the filename for the icu connected component is **filt_topophase.conncomp**.
</div>


In [ ]:
plotdata('merged/filt_topophase.unw',2,title="UNW FILT IFG [rad] ",colormap='jet',colorbar_orientation="vertical")
plotdata('merged/filt_topophase.unw.conncomp',1,title="UNW CONN COMP [-] ",colormap='jet',colorbar_orientation="vertical")

### 3.19 Step unwrap2stage

There is also a two-stage unwrapper where the outputs of the connected component file are used to adjust the unwrapped interferogram. During this optimization procedure, the phase offset between connected component boundaries are minimized. This step can only be run if the unwrapping was done with the  *snaphu_mcf* unwrapper, and also requires the **"do unwrap2stage"** property to be set to *True*. Both are controlled in the topsApp.xml. 

We did not enable the two stage unwrapper. The impact should be small given only small scale connected components.  Executing the **unwrap2stage** will therefore complete instantly.

In [ ]:
!topsApp.py --dostep=unwrap2stage

### 3.20 Step geocode

In the next step we will **geocode** many of the output products created to this point. By default topsApp.py geocodes the following files: **filt_topophase.flat**, **filt_topophase.unw**, **los.rdr**, **phsig.cor**, **topophase.cor** and **topophase.flat**. The user can control which files are geocoded using the **"geocode list"** property in **topsApp.xml**.

In [ ]:
!topsApp.py --dostep=geocode

In [ ]:
plotdata('merged/filt_topophase.unw.geo',2,title="UNW GEO FILT IFG [rad] ",colormap='jet',colorbar_orientation="vertical",datamin=-50, datamax=190)
#plotdata('merged/filt_topophase_2stage.unw.geo',2,title="UNW 2-stage GEO FILT IFG [rad] ",colormap='jet',colorbar_orientation="vertical",datamin=-40, datamax=200)

#### Changing your geocode DEM:

To experiment with a coarser resolution DEM, proceed with this section. By default, the geocoding in topsApp.py is done at the same sampling as processing DEM. However, a different DEM can be specified using the **geocode demfilename** property. 

##### Option 1: Downsample your processing DEM

The ISCE contrib folder has a script called **downsampleDEM.py** that allows you to downsample a DEM to a user-defined sampling. To see how to call the function type: 

In [ ]:
!downsampleDEM.py -h

For example let us downsample the processing DEM that you have been using to 120m resolution. First find out what the name and the path of the DEM is. Either you are using the DEM as specified in the topsApp.xml file or a DEM has been downloaded to your local directory as part of the processing.

In [ ]:
if not os.path.isfile('demLat_N33_N36_Lon_E045_E047.dem.wgs84'):
    SRC_processing_DEM_dir = processing_DEM_dir
    processing_DEM ="demLat_N33_N36_Lon_E045_E047.dem.wgs84"
else:
    SRC_processing_DEM_dir = '.'
    processing_DEM = "demLat_N33_N36_Lon_E045_E047.dem.wgs84"
# making the paths absolute for dir changes
SRC_processing_DEM_dir = os.path.abspath(SRC_processing_DEM_dir)
# providing results to user
print("Your processing DEM is called " + processing_DEM )
print("Stored at: " + SRC_processing_DEM_dir)


Let us now locally downsample this dem to 120m using **downsampleDEM.py**

In [ ]:
cmd = "downsampleDEM.py -i " + processing_DEM + ".vrt -r 120"
print(cmd)

os.chdir(SRC_processing_DEM_dir)
os.system(cmd)

expected_DEM = os.path.join(SRC_processing_DEM_dir,"Coarse_" + processing_DEM)
if os.path.isfile(expected_DEM):
    print("Downsampled DEM:")
    print(expected_DEM)
os.chdir(processing_dir)

Now update your **geocode demfilename** in the topsApp.xml file with the new coarse DEM. Once done, you can run the geocode step again:

In [ ]:
!topsApp.py --dostep=geocode
plotdata('merged/filt_topophase.unw.geo',2,title="UNW GEO FILT IFG [rad] ",colormap='jet',colorbar_orientation="vertical")

##### Option 2: Use of a different DEM

A 3-arc second DEM has been provided with this tutorial. You can use this to geocode at ~90 m resolution by rerunning **geocode** after completing the following two steps:

1) The processing information has already been loaded in the **startup** step through the **topsProc.xml** file, so the geocode DEM is already stored. We will therefore need to manually update the geocode DEM in it. It is a good practice to set it directly in the topsApp.xml file with the property for the *geocode demFilename*.

``` vim
    vim topsProc.xml
```    
    
Remember the xml guidelines:
``` xml
    <Dem_Used>/Users/dbekaert/UNAVCO/TopsApp/DEM/DEM1/demLat_N33_N36_Lon_E045_E047.dem.wgs84</Dem_Used>
        to
    <Dem_Used>../../DEM/DEM3/Coarse_demLat_N33_N36_Lon_E045_E047.dem.wgs84</Dem_Used>
```

2) Update the DEM metadata information. The DEM metadata such as the DEM path is read from the xml information. As the DEM was copied from another machine we need to update the path. fixImageXml.py allows you to do this from the command line.



In [ ]:
!fixImageXml.py -h

From the help, you can see that the **-f** option will update the path in the xml file with an absolute path. Now run the command:

In [ ]:
os.chdir(geocoding_DEM_dir)
!fixImageXml.py -i Coarse_demLat_N33_N36_Lon_E045_E047.dem.wgs84 -f
os.chdir(processing_dir)

Now rerun the **geocode** step again. Do you see the decrease in file size when plotting the result?

In [ ]:
!topsApp.py --dostep=geocode
plotdata('merged/filt_topophase.unw.geo',2,title="UNW GEO FILT IFG [rad] ",colormap='jet',colorbar_orientation="vertical")

### 3.21 Step denseoffsets

In addition to the interferometric processing, we also requested to run the **denseoffsets** step in the **topsApp.xml**. The azimuth and range offsets are estimated by doing a cross-correlation between the coregistered master and slave SLC's on a dense grid, providing a low resolution image of displacement in the range and azimuth domain.

In [ ]:
!topsApp.py --dostep=denseoffsets

The output file, **dense_offsets.bil**, is a two-band file containing the azimuth and range offsets, placed in the "*merged*" folder . The correlation coefficient is contained in the **dense_offsets_snr.bil** file.

Here is a plot of the azimuth and range offsets:

In [ ]:
plotdata('merged/dense_offsets.bil',2,title="DENSE RANGE OFFSETS [pixels] ",colormap='jet',datamin=-0.5, datamax=0.5,colorbar_orientation="vertical")
plotdata('merged/dense_offsets.bil',1,title="DENSE AZIMUTH OFFSETS [pixels] ",colormap='jet',datamin=-0.25, datamax=0.25,colorbar_orientation="vertical")

### 3.22 Step filteroffsets

The denseoffsets can be a bit noisy. In the following step, we will clean this up by applying a filter. The output will be added to the "*merged*" folder under the filename **filt_dense_offsets.bil**.

In [ ]:
!topsApp.py --dostep=filteroffsets

### 3.23 Step geocodeoffsets

Geocoding the filtered dense offsets generates a **filt_dense_offsets.bil.geo** file in the "*merged*" folder.

In [ ]:
!topsApp.py --dostep=geocodeoffsets

Here is a plot of the final geocoded filtered dense offsets:

In [ ]:
plotdata('merged/filt_dense_offsets.bil.geo',2,title="DENSE AZIMUTH OFFSETS [pixels] ",colormap='jet',datamin=-0.5, datamax=0.5,colorbar_orientation="vertical")
plotdata('merged/filt_dense_offsets.bil.geo',1,title="DENSE RANGE OFFSETS [pixels] ",colormap='jet',datamin=-0.25, datamax=0.25,colorbar_orientation="vertical")

# Relevant references:
- Heresh Fattahi, Piyush Agram, and Mark Simons, *Precise coregistration of Sentinel-1A TOPS data*, https://files.scec.org/s3fs-public/0129_1400_1530_Fattahi.pdf
- ESA, *Sentinel-1 and TOPS overview*, https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar
- Nestor Yague-Martinez, Pau Prats-Iraola, Fernando Rodriguez Gonzalez,Ramon Brcic, Robert Shau, Dirk Geudtner, Michael Eineder, and Richard Bamler, *Interferometric Processing of Sentinel-1 TOPS Data*, IEEE, doi:10.1109/TGRS.2015.2497902, https://ieeexplore.ieee.org/document/7390052/
